In [19]:
import digitalio
import board
import busio
import adafruit_rfm9x
import json
import numpy as np
import random
import pypuf.io
import pypuf.simulation
import time
import hashlib
import pickle
import zlib
from digitalio import DigitalInOut, Direction, Pull
import adafruit_ssd1306



RADIO_FREQ_MHZ = 433.0
CS = digitalio.DigitalInOut(board.CE1)
RESET = digitalio.DigitalInOut(board.D25)

spi = busio.SPI(board.SCK, MOSI=board.MOSI, MISO=board.MISO)
rfm9x = adafruit_rfm9x.RFM9x(spi, CS, RESET, RADIO_FREQ_MHZ)

rfm9x.tx_power = 23
# enable CRC checking
rfm9x.enable_crc = True
# set delay before transmitting ACK (seconds)
rfm9x.ack_delay = 0.1
# set node addresses
rfm9x.node = 2
rfm9x.destination = 1
rfm9x.ack_retries = 5
rfm9x.ack_wait = 2.0  # 2 seconds


puf_BITS = 32
puf_SEED = 1
puf_SETS = 8

def send_message(key, value):
    msg = f"Sending {key} from Key to Car..."
    print(msg)
    if isinstance(value, str):
        value = value.encode('utf-8')
    rfm9x.send_with_ack(value)
    
    print(f"{key} has been sent")


def receive_message(key):
    print(f"Receiving {key} from Key to Car...")
    MAX_CHUNK_SIZE = 170  # Adjusted chunk size to match send_message
    received_data = b''
    while True:
        data = rfm9x.receive(with_ack=True)
        if data is not None:
            received_data += data
            # If we received less than the maximum chunk size, we've received the last chunk
            if len(data) < MAX_CHUNK_SIZE:  
                break
        time.sleep(0.1)  # Add a small delay before checking again
    print(f"Received {key}")
    return received_data


# Access the values
key_id = bin(random.getrandbits(puf_BITS))[2:].zfill(puf_BITS)

Ks = bin(random.getrandbits(puf_BITS))[2:].zfill(puf_BITS) #Generate Ks
Ks_ndarray = np.array([int(c) for c in Ks], dtype=np.int8) #type: string-> numpy array

"""
Generate & XOR Ni
"""
Ni = bin(random.getrandbits(puf_BITS))[2:].zfill(puf_BITS) #type: string

Ni_ndarray = np.array([int(c) for c in Ni], dtype=np.int8) #type: string-> numpy array

Ni_encrypted = Ks_ndarray ^ Ni_ndarray #type: numpy array

"""
# Send MA1
"""

# Create a dictionary to hold both pieces of data
message_data = {
    'ID': key_id,
    'Encrypted_Ni': Ni_encrypted.tobytes().hex()  # Convert bytes to hex string for JSON serialization
}

# Convert the dictionary to a JSON string
message_json = json.dumps(message_data)

# Send the combined message
send_message('MA1_data', message_json)


Sending MA1_data from Key to Car...
MA1_data has been sent
